In [1]:
import Pkg; 
cd(joinpath(@__DIR__, "../../"))
Pkg.activate("Project.toml")

using MorphoMol
using StaticArrays
using Rotations
using Distances
using JLD2
using GLMakie

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [2]:
folder = "../../Data/hpc_out/rwm_wp_2_6r7m/"

id_index = 1
max = 1
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        max = max < parse(Int, split(file, "_")[id_index]) ? parse(Int, split(file, "_")[id_index]) : max
    end
end

min_Es = [Inf for _ in 1:max]
evaluation_strings = ["" for _ in 1:max]
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        @load "$folder$file" input output
        id = parse(Int, split(file, "_")[id_index])
        string = "$(id): $(input["T"]) | $(output["αs"][end]) | $(output["Es"][1]), $(minimum(output["Es"])), $(output["Es"][end])"
        evaluation_strings[id] = string
        min_Es[id] = minimum(output["Es"])
    end
end

In [3]:
println(argmin(min_Es))
for elem in evaluation_strings
    println(elem)
end

61

2: 2.0 | 0.004517182 | 8286.307403868293, 8164.712056984222, 8177.025581422826
3: 2.0 | 0.009021854 | 8219.535117314525, 8164.302964245558, 8164.302964245558
4: 2.0 | 0.0718444 | 8218.809186151842, 8157.415378652751, 8167.420576413768
5: 2.0 | 0.004092769 | 8199.651271836045, 8150.911551009964, 8154.53919318752
6: 2.0 | 0.07835584 | 8215.664497652384, 8161.606442666041, 8172.712357489147
7: 2.0 | 0.04889637 | 8219.57830446917, 8160.517090664551, 8176.732219889337
8: 2.0 | 0.009959503 | 8217.19031795141, 8158.004960463167, 8158.004960463167
9: 2.0 | 0.063833825 | 8210.381748192185, 8161.461711847608, 8170.965314148354
10: 2.0 | 0.06131399 | 8219.568405820808, 8161.562583246138, 8167.513917523415
11: 2.0 | 0.051430084 | 8219.578304347451, 8161.9736749835865, 8171.923742331928
12: 2.0 | 0.051100284 | 8218.967451175533, 8160.062779177901, 8171.898957763464
13: 2.0 | 0.008546494 | 8507.258672406218, 8159.755607494894, 8169.636865280451
14: 2.0 | 0.085323766 | 8217.44427954127, 8163.1963

In [4]:
function interactive_simulation_analysis(input, output)
    dgms = output["IDGMs"]
    Es = output["Es"]
    persistence_weight = -0.1
    realizations = [MorphoMol.Utilities.get_matrix_realization(state, input["template_centers"]) for state in output["states"]]
    tps = [MorphoMol.Energies.get_total_persistence(dgm) for dgm in dgms]
    cdgms2 = [dgm[2] for dgm in dgms]
    cdgms3 = [dgm[3] for dgm in dgms]
    cp2s = [MorphoMol.Energies.get_persistence(dgm) for dgm in cdgms2]
    cp3s = [MorphoMol.Energies.get_persistence(dgm) for dgm in cdgms3]
    mol_type = input["mol_type"]
    exp_template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["template_centers"]
    exp_state = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["state"]
    thetas = [MorphoMol.Utilities.average_offset_distance(exp_template_centers, input["template_centers"], exp_state, state) for state in output["states"]]
    xs = [i for i in 1:length(dgms)];
    two_ps = [MorphoMol.Energies.get_persistence(dgm[2], -0.1) for dgm in dgms]
    three_ps = [MorphoMol.Energies.get_persistence(dgm[3], -0.1) for dgm in dgms]
    tps = [MorphoMol.Energies.get_total_persistence(dgm, -0.1) for dgm in dgms];

    f = Figure(fontsize = 7)

    #Slider
    sl_i = Slider(f[4, 1:4], range = 1:length(dgms), startvalue = 1)
    x = sl_i.value

    # Persistence Diagrams
    pd_ax_a = Axis(f[1, 1], title = "Persistence codim = 2", xticks = 0:25:100, yticks = 0:25:100)
    pd_ax_b = Axis(f[1, 2], title = "Persistence codim = 3", xticks = 0:25:100, yticks = 0:25:100)
    dgm_points_2 = @lift([Point2f(dgms[$x][2][i,1], dgms[$x][2][i,2]) for i in 1:size(dgms[$x][2])[1]])
    dgm_points_3 = @lift([Point2f(dgms[$x][3][i,1], dgms[$x][3][i,2]) for i in 1:size(dgms[$x][3])[1]])

    ms = 5
    scatter!(pd_ax_a, dgm_points_2, color = :orange, markersize = ms)
    lines!(pd_ax_a, [0, 100], [0, 100], color=:black)
    scatter!(pd_ax_b, dgm_points_3, color = :green, markersize = ms)
    lines!(pd_ax_b, [0, 100], [0, 100], color=:black)

    #Energy and Theta
    theta_ax = Axis(f[2, 1], title = L"\Theta")
    E_ax = Axis(f[2, 2], title = L"F_{sol}")

    theta_mark = @lift(Point2f($x, $@lift(thetas[$x])))
    E_mark = @lift(Point2f($x, $@lift(Es[$x])))

    scatter!(theta_ax, xs, thetas, color=:magenta, markersize = ms)
    scatter!(theta_ax, theta_mark, color=:black, markersize = 10)

    scatter!(E_ax, xs, Es, color=:blue, markersize = ms)
    scatter!(E_ax, E_mark, color=:black, markersize = 10)


    # Configuration
    conf_ax = Axis3(f[1:2, 3:4])
    c1_points = @lift([Point3f(realizations[$x][1][1,i], realizations[$x][1][2,i], realizations[$x][1][3,i]) for i in 1:size(realizations[$x][1])[2]])
    c2_points = @lift([Point3f(realizations[$x][2][1,i], realizations[$x][2][2,i], realizations[$x][2][3,i]) for i in 1:size(realizations[$x][2])[2]])

    conf_ms = 10
    scatter!(conf_ax, c1_points, markersize = conf_ms)
    scatter!(conf_ax, c2_points, markersize = conf_ms)

    #Persistence Measures
    two_p_ax = Axis(f[3, 1], title = "codim 2 persistence")
    three_p_ax = Axis(f[3, 2], title = "codim 3 persistence")
    tp_ax = Axis(f[3, 3], title = "Total Persistence")

    tp_mark = @lift(Point2f($x, $@lift(tps[$x])))
    two_p_mark = @lift(Point2f($x, $@lift(two_ps[$x])))
    three_p_mark = @lift(Point2f($x, $@lift(three_ps[$x])))

    scatter!(tp_ax, xs, tps, color=:red, markersize = ms)
    scatter!(tp_ax, tp_mark, color = :black, markersize = 10)

    scatter!(two_p_ax, xs, two_ps, color=:orange, markersize = ms)
    scatter!(two_p_ax, two_p_mark, color=:black, markersize = 10)

    scatter!(three_p_ax, xs, three_ps, color=:green, markersize = ms)
    scatter!(three_p_ax, three_p_mark, color=:black, markersize = 10)
    f
end


interactive_simulation_analysis (generic function with 1 method)

In [5]:
@load "$(folder)4_rwm_wp_2_6r7m.jld2" input output
display(interactive_simulation_analysis(merge!(input, Dict("mol_type" => "6r7m")), output))

GLMakie.Screen(...)